In [1]:
# Packages
install.packages("feather", quiet = TRUE)

# Librairies
library(aws.s3)
library(data.table)
library(feather)

# Fonctions
import_feather <- function(input_file) {
  bucket_name <- "maeldieudonne"
  data <- aws.s3::s3read_using(
    FUN = arrow::read_feather,
    object = input_file,
    bucket = bucket_name,
    opts = list("region" = "")
  )  
  data <- as.data.table(data)
  return(data)
}

<h1>Résultats électoraux</h1>

plm = 0-1-2 (0 : communes non PLM ; 1 : 45 arrondissements PLM ; 2 : 3 communes entières PLM)  
codecommune2 = code de la nouvelle commune de rattachement ou de substitution (au cas statut1999=2,3,4) ou de la commune actuelle (au cas statut1999=1). Le fichier comprend plusieurs lignes par codecommune2 dès lors que plusieurs communes ont fusionné en une seule entre 1943 et 1999.  
dep2 = code du nouveau département de rattachement ou de substitution (au cas statut1999=2,3,4) ou du département actuel (au cas statut1999=1)

In [2]:
pres1981 <- import_feather("/diffusion/Elections_pres/pres1981comm.feather")

# Recodages
candidats <- fread("candidats.csv", header = TRUE)

## Participation
pres1981[, part := votants/inscrits]
pres1981[, partT2 := votantsT2/inscritsT2]

## Droite / gauche
T1mask <- candidats[DrGau == "G", .(variable = paste0("voix", Candidat))]
pres1981[, GD_ratio_T1 := rowSums(.SD, na.rm = TRUE) / exprimes, .SDcols = T1mask$variable]
T2mask <- candidats[DrGau == "G", .(variable = paste0("voixT2", Candidat))]
T2candidates <- intersect(T2mask$variable, names(pres1981))
pres1981[, GD_ratio_T2 := rowSums(.SD, na.rm = TRUE) / exprimesT2, .SDcols = T2candidates]

## Libéralisme culturel / conservatisme moral
T1mask <- candidats[LibCons == "L", .(variable = paste0("voix", Candidat))]
pres1981[, lib_ratio_T1 := rowSums(.SD, na.rm = TRUE) / exprimes, .SDcols = T1mask$variable]

# Nettoyage
prefixes_to_drop <- c("exprimes", "inscrits", "nom", "pervote", "ppar", "pvoix", "pvote", "voix", "vot")
pres1981 <- pres1981[, .SD, .SDcols = setdiff(names(pres1981), grep(paste0("^", paste(prefixes_to_drop, collapse = "|")), names(pres1981), value = TRUE))]
pres1981 <- pres1981[, -c("nuls", "plmdoublon", "perpar", "year")]

<h1>Propriété immobilière</h1>

npropri81 = nombre de ménages propriétaires dans la commune  
nlogement1981 = nombre total de logements dans la commune (propriétaires, locataires ou logés gratuitement) (résidences principales)  
ppropri1981 = proportion de ménages propriétaires de leur logement dans la commune

In [3]:
propri <- import_feather("/diffusion/Proprietaires/proprietairescommunes.feather")

<h1>Contrôles</h1>
<h2>1. Population</h2>

pop1981  
popagglo1981

In [4]:
popul <- import_feather("/diffusion/Taille_agglo_commune/popcommunes.feather")

<h2>2. Âge</h2>

age1981 = âge moyen de la commune  
prop0141981 = proportion de 0-14 ans dans la commune  
prop15391981 = proportion de 15-39 ans dans la commune  
prop40591981 = proportion de 40-59 ans dans la commune  
prop60p1981 = proportion de 60 ans et + dans la commune

In [5]:
ages <- import_feather("/diffusion/Age_csp/agesexcommunes.feather")

<h2>3. Diplômes</h2>

nodip1981 = nombre total de personnes 25+ sans diplôme ou dont le diplôme le plus élevé est le BEPC, le brevet des collèges, le BEP ou le CAP  
bac1981 = nombre total de personnes 25+ dont le diplôme le plus élevé est le baccalauréat général ou technologique  
sup1981 = nombre total de personnes 25+ diplômées du supérieur  

In [6]:
diplo <- import_feather("/diffusion/Diplomes/diplomescommunes.feather")

# Rapporter nodip1981, bac1981 et sup1981 au nombre d'habitants.

<h2>4. Religiosité</h2>

prefract1791 = proportion de prêtres réfractaires en 1791  
nclerge1791 = nombre total des prêtres soumis au serment de 1791

pclerge1856 = effectifs totaux du clergé (prêtres, religieux, religieuses) exprimés en proportion de la population totale  
pmessalisants1950 = effectifs totaux de personnes allant à la messe du dimanche en 1950 en proportion de la population totale

In [7]:
religcomm <- import_feather("/diffusion/Enseignement_prive/religiositecommunes.feather")

# Recodages
# nclerge1791 à rapporter à la taille de la population pour avoir une idée de la présence des institutions religieuses dans la commune.

In [8]:
religdep <- import_feather("/diffusion/Enseignement_prive/religiositedepartements.feather")

<h2>5. CSP</h2>

pagri1981 = proportion d’agriculteurs parmi les actifs 25-54 ans (=agri/pact)  
pindp1981 = proportion d’indépendants (artisans, commerçants, chefs d’entreprises) parmi les actifs 25-54 ans (=indp/pact)  
pcadr1981 = proportion de cadres et professions intellectuelles supérieures parmi les actifs 25-54 ans (=cadr/pact)  
ppint1981 = proportion de professions intermédiaires parmi les actifs 25-54 ans (=pint/pact)  
pempl1981 = proportions d’employés parmi les actifs 25-54 ans (=empl/pact)  
pouvr1981 = proportion d’ouvriers parmi les actifs 25-54 ans (=ouvr/pact)  
pchom1981 = proportion de chômeurs parmi les actifs 25-54 ans (=chom/pact)

In [9]:
catsociopro <- import_feather("/diffusion/CSP/cspcommunes.feather")

<h2>6. Nationalités</h2>

petranger1981 = proportion de personnes de nationalité étrangère

In [10]:
nation <- import_feather("/diffusion/Nationalites/etrangerscommunes.feather")

<h2>7. Productions et revenus</h2>

revmoy1981 = revenu moyen par habitant de la commune (exprimé en euros de 2022)

In [11]:
reven <- import_feather("/diffusion/Revenus/revcommunes.feather")

<h1>Construction de la base</h1>

In [12]:
pres1981[propri, c("ppropri1981") :=
             .(i.ppropri1981), on = "codecommune"]
pres1981[popul, c("pop1981", "popagglo1981") :=
             .(i.pop1981, i.popagglo1981), on = "codecommune"]
pres1981[ages, c("age1981", "prop0141981", "prop15391981", "prop40591981", "prop60p1981") :=
             .(i.age1981, i.prop0141981, i.prop15391981, i.prop40591981, i.prop60p1981), on = "codecommune"]
# Diplômes
pres1981[religcomm, c("prefract1791") :=
             .(i.prefract1791), on = "codecommune"]
pres1981[religdep, c("pclerge1856", "pmessalisants1950") :=
             .(i.pclerge1856, i.pmessalisants1950), on = "dep"]
pres1981[catsociopro, c("pagri1981", "pindp1981", "pcadr1981", "ppint1981", "pempl1981", "pouvr1981", "pchom1981") :=
             .(i.pagri1981, i.pindp1981, i.pcadr1981, i.ppint1981, i.pempl1981, i.pouvr1981, i.pchom1981), on = "codecommune"]
pres1981[nation, c("petranger1981") :=
             .(i.petranger1981), on = "codecommune"]
pres1981[reven, c("revmoy1981") :=
             .(i.revmoy1981), on = "codecommune"]

In [13]:
write_feather(pres1981, "pres1981.feather")